In [38]:
#get env setup
import getpass
import os
from dotenv import load_dotenv

#get env setup
load_dotenv('nb.env', override=True)

if not os.environ.get('NEO4J_URI'):
    os.environ['NEO4J_URI'] = getpass.getpass('NEO4J_URI:\n')
if not os.environ.get('NEO4J_USERNAME'):
    os.environ['NEO4J_USERNAME'] = getpass.getpass('NEO4J_USERNAME:\n')
if not os.environ.get('NEO4J_PASSWORD'):
    os.environ['NEO4J_PASSWORD'] = getpass.getpass('NEO4J_PASSWORD:\n')

NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')

In [63]:
import json
from person import Person

#read json models back
with open('extracted-people-data.json', 'r') as file:
    people_json = json.load(file)
people = [Person(**person) for person in people_json]
people[0]

Person(id='5BiANRmk', name='Alex Thompson', email='alex.thompson@email.com', current_title='Junior Mobile Developer', department=<Department.ENGINEERING: 'Engineering'>, level=<Level.JUNIOR: 'Junior'>, hire_date=None, skills=[HasSkill(skill=Skill(name=<SkillName.JAVASCRIPT: 'JavaScript'>), proficiency=3, years_experience=2, context='Used in personal projects and during mobile development internship.', is_primary=False), HasSkill(skill=Skill(name=<SkillName.PYTHON: 'Python'>), proficiency=None, years_experience=None, context=None, is_primary=False), HasSkill(skill=Skill(name=<SkillName.JAVA: 'Java'>), proficiency=1, years_experience=None, context='Basic familiarity from coursework.', is_primary=False), HasSkill(skill=Skill(name=<SkillName.DATA_ENGINEERING: 'Data Engineering'>), proficiency=None, years_experience=None, context=None, is_primary=False), HasSkill(skill=Skill(name=<SkillName.PRODUCT_MANAGEMENT: 'Product Management'>), proficiency=None, years_experience=None, context=None, is

In [64]:
from neo4j import GraphDatabase

# load into People nodes in Neo4j

#instantiate driver
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

#test neo4j connection
driver.execute_query("MATCH(n) RETURN count(n)")

EagerResult(records=[<Record count(n)=10>], summary=<neo4j._work.summary.ResultSummary object at 0x10dc02bd0>, keys=['count(n)'])

In [65]:
from neo4j import RoutingControl

#create uniqueness constraint if not exists
driver.execute_query(
    'CREATE CONSTRAINT IF NOT EXISTS FOR (n:Person) REQUIRE (n.id) IS NODE KEY',
    #database_=DATABASE,
    routing_=RoutingControl.WRITE
)

driver.execute_query(
    'CREATE CONSTRAINT IF NOT EXISTS FOR (n:Skill) REQUIRE (n.name) IS NODE KEY',
    #database_=DATABASE,
    routing_=RoutingControl.WRITE
)

driver.execute_query(
    'CREATE CONSTRAINT IF NOT EXISTS FOR (n:Thing) REQUIRE (n.name) IS NODE KEY',
    #database_=DATABASE,
    routing_=RoutingControl.WRITE
)

driver.execute_query(
    'CREATE CONSTRAINT IF NOT EXISTS FOR (n:Domain) REQUIRE (n.name) IS NODE KEY',
    #database_=DATABASE,
    routing_=RoutingControl.WRITE
)

driver.execute_query(
    'CREATE CONSTRAINT IF NOT EXISTS FOR (n:WorkType) REQUIRE (n.name) IS NODE KEY',
    #database_=DATABASE,
    routing_=RoutingControl.WRITE
)


EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x10dbba290>, keys=[])

In [76]:
# merge people
def chunks(xs, n=10):
    n = max(1, n)
    return [xs[i:i + n] for i in range(0, len(xs), n)]

for chunk in chunks(people_json):
    records = driver.execute_query(
        """
        UNWIND $records AS rec
        MERGE(person:Person {id:rec.id})
        SET person.name = rec.name,
            person.email = rec.email,
            person.current_title = rec.current_title,
            person.department = rec.department,
            person.level = rec.level,
            person.years_experience = rec.years_experience,
            person.location = rec.location
        RETURN count(rec) AS records_upserted
        """,
        #database_=DATABASE,
        routing_=RoutingControl.WRITE,
        result_transformer_= lambda r: r.data(),
        records = chunk
    )
    print(records)

[{'records_upserted': 10}]


In [77]:
# merge accomplishments
skills = []
accomplishments = []
for person in people_json:

    # extend skills list
    tmp_skills = person['skills'].copy()
    for skill in tmp_skills:
        skill['personId'] = person['id']
    skills.extend(tmp_skills)

    # extend accomplishments list
    tmp_accomplishments = person['accomplishments'].copy()
    for accomplishment in tmp_accomplishments:
        accomplishment['personId'] = person['id']
    accomplishments.extend(tmp_accomplishments)



In [68]:
skills[:3]

[{'skill': {'name': 'JavaScript'},
  'proficiency': 3,
  'years_experience': 2,
  'context': 'Used in personal projects and during mobile development internship.',
  'is_primary': False,
  'personId': '5BiANRmk'},
 {'skill': {'name': 'Python'},
  'proficiency': None,
  'years_experience': None,
  'context': None,
  'is_primary': False,
  'personId': '5BiANRmk'},
 {'skill': {'name': 'Java'},
  'proficiency': 1,
  'years_experience': None,
  'context': 'Basic familiarity from coursework.',
  'is_primary': False,
  'personId': '5BiANRmk'}]

In [69]:
accomplishments[:2]

[{'type': 'BUILT',
  'thing': {'name': 'banking_app_5BiANRmk',
   'type': 'PRODUCT',
   'domain': 'MOBILE'},
  'impact_description': 'Developed a secure banking mobile app using Swift during a 6-month internship, implemented secure authentication.',
  'year': 2024,
  'role': 'Junior Mobile Developer',
  'duration': '6 months',
  'team_size': None,
  'context': 'FinTech Mobile internship',
  'personId': '5BiANRmk'},
 {'type': 'BUILT',
  'thing': {'name': 'social_media_ios_app_5BiANRmk',
   'type': 'PRODUCT',
   'domain': 'MOBILE'},
  'impact_description': 'Built a social media iOS application as a capstone project, supporting photo sharing and messaging.',
  'year': 2023,
  'role': 'Mobile Development Intern',
  'duration': '3 months',
  'team_size': None,
  'context': 'Social Media Startup internship',
  'personId': '5BiANRmk'}]

In [78]:
for chunk in chunks(skills):
    records = driver.execute_query(
        """
        UNWIND $records AS rec
        MATCH(person:Person {id:rec.personId})
        MERGE(skill:Skill {name:rec.skill.name})
        MERGE(person)-[r:KNOWS]->(skill)
        SET r.proficiency = rec.proficiency,
            r.years_experience = rec.years_experience,
            r.context  = rec.context,
            r.is_primary = rec.is_primary
        RETURN count(rec) AS records_upserted
        """,
        #database_=DATABASE,
        routing_=RoutingControl.WRITE,
        result_transformer_= lambda r: r.data(),
        records = chunk
    )
    print(records)

[{'records_upserted': 10}]
[{'records_upserted': 10}]
[{'records_upserted': 10}]
[{'records_upserted': 10}]
[{'records_upserted': 10}]
[{'records_upserted': 10}]
[{'records_upserted': 10}]
[{'records_upserted': 10}]
[{'records_upserted': 4}]


In [79]:
for chunk in chunks(accomplishments):
    records = driver.execute_query(
        """
        UNWIND $records AS rec

        //match people
        MATCH(person:Person {id:rec.personId})

        //merge accomplishments
        MERGE(thing:Thing {name:rec.thing.name})
        MERGE(person)-[r:$(rec.type)]->(thing)
        SET r.impact_description = rec.impact_description,
            r.year = rec.year,
            r.role  = rec.role,
            r.duration = rec.duration,
            r.team_size = rec.team_size,
            r.context  = rec.context

        //merge domain and work type
        MERGE(Domain:Domain {name:rec.thing.domain})
        MERGE(thing)-[:IN]->(Domain)
        MERGE(WorkType:WorkType {name:rec.thing.type})
        MERGE(thing)-[:OF]->(WorkType)

        RETURN count(rec) AS records_upserted
        """,
        #database_=DATABASE,
        routing_=RoutingControl.WRITE,
        result_transformer_= lambda r: r.data(),
        records = chunk
    )
    print(records)

[{'records_upserted': 10}]
[{'records_upserted': 10}]
[{'records_upserted': 10}]
[{'records_upserted': 10}]
[{'records_upserted': 10}]
[{'records_upserted': 10}]
[{'records_upserted': 10}]


In [ ]:
# build adk agent with neo4j mcp

In [ ]:
# ask some questions